## Noa project

The goal is to convert the Program object into a DreamCoder program, which is expressed via lambda calculus. That way we can run DreamCoder's compression algorithm on the programs we create from our bidirectional search algorithm. You can manually construct the program by composing different classes together from `dreamcoder/program.py`. There should be a way to check that the program you create, when evaluated on the input grids, generates the output grids. These should be written as automatic tests done to make sure that the creations are correct.

In [ ]:
from bidir.primitives.functions import Function
from ec.dreamcoder.program import Primitive

First, you have to make the primitive functions:

In [1]:
def convert_to_dc_primitive(fn: Function) -> Primitive:
    raise NotImplementedError

NameError: name 'Function' is not defined

For example, we might want to convert `rotate_cw` and `block` into corresponding DC primitives, copied below for reference:

In [2]:
def rotate_ccw(grid: Grid) -> Grid:
    return Grid(np.rot90(grid.arr), grid.pos)


def block(height: int, width: int, color: Color) -> Grid:
    """Returns a solid-colored grid of given shape."""
    arr = np.full((height, width), color.value)
    return Grid(arr)

NameError: name 'Grid' is not defined

To do this, we can do something like this:

In [3]:
from ec.dreamcoder.type import arrow
tgrid = baseType("grid")
tint = baseType("tint")
tcolor = baseType("tcolor")


rotate_cw_dc = Primitive("rotate_cw", arrow(tgrid, tgrid), lambda grid: rotate_cw(grid))
block_dc = Primitive("block", arrow(tint, tint, tcolor, tgrid), lambda height: lambda w: lambda color: block(height, width, color))

ModuleNotFoundError: No module named 'ec'

Steps for you to work on:
1. Take the code written above for creating a rotate_cw primitive, and make sure it runs correctly. 
2. Modify the arc input so that the data types are the same as what's used for the RL set-up.
3. Modify the arc demo (or another DC script I gave you) so that it tries solving tasks using the rotate_cw primitive here. Should be able to solve a couple of the symmetry tasks that only involve rotations.
4. Implement a few more primitives in the same way.
5. The goal is to convert all of the RL primitives into DC primitives. This can be done automatically, if you notice the pattern in how the name, arrow type, and lambda function at the end is made. Implement this function. You should take a bidir.primitives.functions.Function as input, as this stores the type info for an RL primitive.

Then you can work on automatically converting an RL Program into a Dreamcoder Program, by taking the Program object, converting all of the function calls into DC function calls, and converting the Program AST into the lambda calculus style thing that I showed you before (do you still have that code?

Step 2 will probably be confusing, because you have to understand how some of the Dreamcoder code is organized. Here are some steps to guide you along understanding how it is organized:
1. Run the ARC demo.
2. Look into where the tasks are created. How to we input ARC tasks to Dreamcoder? The type is `Task`, from `dreamcoder/task.py`. How are these tasks created? Check out dreamcoder/domains/arc/makeTasks.py.
3. We'd like to create tasks in DC so that they're created out of the same Grid type as the RL tasks. What type is currently used for representing ARC tasks? It should be the Grid type from dreamcoder/domains/arc/arcPrimitives.py. Instead of using this Grid type, we'd like to use the Grid type from bidir/primitives/types.py. 
4. Make a new function that, given a task number, creates an ARC task using the RL Grid type. Note you have to specify the type of the task, i.e. the input and output. it should be arrow(tgrid, tgrid): one input grid, one output grid. You can reuse the tgrid type from before, although instead of importing it from arcPrimitives, I would redefine a new copy of it, so your code is separate from arcPrimitives. (`tgrid = basetype("grid")`).

Once you do these steps, you can move on to step 3. Now, since the task uses the underlying Grid type from the RL framework, we can use our primitive that wraps an RL primitive into a DC primitive, and so the primitive operates on the right type.

Step four is just to notice the pattern of how these DC-wrapper primitives are made based on the RL primitives. Once you notice the pattern, you can do step 5 to do it automatically for all of the RL primitives.